In [ ]:
from traceback import print_tb
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
import torch.utils.data as data
import torch
import json
import nltk
from tqdm import tqdm
from pathlib import Path
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
# nltk.download('stopwords')
# nltk.download('punkt')
%matplotlib inline
# IMAGE_PATH = Path("/etlstage/PEE_joint/mine/image_data")
# %%
nltk.data.path.append("/home/hdfsf10n/nltk_data")
nltk.data.path
# %%
%load_ext autoreload
%autoreload
from src.utils.data_loader import get_loader
# %%
with open('./data/vaild_dataset.json', 'r') as outfile:
  product_info = pd.read_json(json.load(outfile), orient="records")
print(product_info.shape)
# product_info["imageURLHighRes"] = product_info["file_name"].apply(lambda file_name : IMAGE_PATH/file_name)

import re
def get_space_len(s):
  return len(max(re.findall(' +', s), key=len, default=[0]))


In [ ]:
with open('./data/vaild_dataset.json', 'r') as outfile:
  product_info = pd.read_json(json.load(outfile), orient="records")
  
product_info = product_info[product_info.valid == True]
product_info = product_info[product_info["perCategory"].isin(["AMAZON_FASHION", 'All_Beauty',
                                                               'Toys_and_Games','Office_Products',
                                                              'Home_and_Kitchen','Electronics',
                                                              'Clothing_Shoes_and_Jewelry'])]
product_info["file_name"] = product_info["imageURLHighRes"].str.split("/").str[-1]

product_info = product_info.explode("description")
product_info['description'] = product_info['description'].str.replace(r'<[^<>]*>', '', regex=True)
product_info["description"] = product_info["description"].str.split(r'\;|\!|\?|\.|\-')
product_info = product_info.explode("description")
product_info["description"] = product_info["description"].str.split(r'\n')
product_info = product_info.explode("description")
product_info["description"] = product_info["description"].str.lstrip(r'\,')
# product_info["description"] = product_info["description"].str.split(r'<br>|</td>')
# product_info = product_info.explode("description")

product_info = product_info.where(
    (product_info["description"].str.len()>50) & 
    (product_info["description"].astype(str).apply(get_space_len)<4) & 
    (product_info["description"] != "") &
    (not "<td" in product_info["description"])
    ).dropna().reset_index(drop=True)
product_info["description"] = product_info["description"].str.strip()
product_info['sentence_index'] = product_info.groupby('file_name').cumcount()  # Enumerate Groups
product_info = product_info[product_info['sentence_index'] <= product_info["description"].str.len().min()].reset_index(drop=True)
print(product_info.shape)
product_info.head(5)

# for descript in product_info.loc[:10,"description"]:
#     print("="*100)
#     print(descript)

In [ ]:
import random
idex = random.randint(0, product_info.shape[0])
# img_url = product_info.loc[idex,"imageURLHighRes"]
img_loc = product_info.loc[idex,"imageURLHighRes"]
caption = product_info.loc[idex,"description"]
Category = product_info.loc[idex,["perCategory", "sentence_index"]]
print(caption)
print(Category)
print(img_loc)
I = io.imread(img_loc)
plt.axis('off')
plt.imshow(I)

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
import sys
import os
import math
import pickle
from sklearn.model_selection import train_test_split
import torch.utils.data as data
import nltk
from nltk.translate.bleu_score import corpus_bleu
nltk.download('punkt')

%load_ext autoreload
%autoreload
from src.utils.data_loader import get_loader
from src.models.CNN_Encoder import EncoderCNN
from src.models.RNN_decoder_catAttention import DecoderRNN
from src.models.MLP_Encoder import MlpEncoder
from src.models.trainer import train
from src.models.validator import validate
from src.utils.utils_trainer import get_batch_caps, get_hypothesis, adjust_learning_rate
from torchvision import transforms
from sklearn.preprocessing import OneHotEncoder


In [ ]:
current_time = "2022-04-03-20-32-35"
encoder_file = 'encoder-0.pkl' 
decoder_file = 'decoder-0.pkl'

log_train = f'./logs/{current_time}/training_log.txt'       # name of files with saved training loss and perplexity
log_val = f'./logs/{current_time}/validation_log.txt'
bleu = f'./logs/{current_time}/bleu.txt'
vocab_file = f'./assets/{current_time}/vocab.pkl' 

with open(bleu, 'r') as f: 
    records = f.readlines()
bleu_scores = []
for record in records:
    bleus = []
    for blue in record.split(',')[-4:]:
        bleus.append(float(blue.split(":")[-1].strip()))
    bleu_scores.append(bleus)

with open(f'../models_new/{current_time}/onehot_cat_encoder', "rb") as f:
    onehot_cat_encoder = pickle.load(f)

with open(f'../models_new/{current_time}/onehot_seq_encoder', "rb") as f:
    onehot_seq_encoder = pickle.load(f)

# Open the training log file.
file_train = open(log_train, 'a')
file_val = open(log_val, 'a')
bleu_score_file = open(bleu, 'a')

In [ ]:
train_df, val_test_df = train_test_split(product_info, test_size=0.4)
train_df.reset_index(drop=True,inplace=True)
valid_df, test_df = train_test_split(val_test_df, test_size=0.5)
valid_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

transform_test = transforms.Compose([ 
    transforms.Resize(500),                          # smaller edge of image resized to 256
    transforms.Resize((320,320)),                      # get 224x224 crop from random location
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
])

test_data_loader = get_loader(test_df,
                        transform=transform_test,
                        onehot_cat_encoder=onehot_cat_encoder,
                        mode='test',
                        onehot_seq_encoder=onehot_seq_encoder,
                        image_type="imageURLHighRes",
                        caption_type="description",
                        vocab_file = vocab_file,
                        batch_size=1)

In [ ]:
embed_size = 125           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_features = 2048        # number of feature maps, produced by Encoder
vocab_size = len(test_data_loader.dataset.vocab)

In [ ]:

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN()
encoder.eval()
decoder = DecoderRNN(num_features = num_features, 
                     embedding_dim = embed_size,
                     category_dim = len(onehot_cat_encoder.get_feature_names()) + len(onehot_seq_encoder.get_feature_names()),
                     hidden_dim = hidden_size, 
                     vocab_size = vocab_size,
                     cat_attention=True)
decoder.eval()

# Load the trained weights.
encoder.load_state_dict(torch.load(os.path.join(f'../models_new/{current_time}', encoder_file), map_location='cpu'))
decoder.load_state_dict(torch.load(os.path.join(f'../models_new/{current_time}', decoder_file), map_location='cpu'))

device = "cuda"

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)

In [ ]:
def clean_sentence(output, data_loader):
    vocab = data_loader.dataset.vocab.idx2word
    words = [vocab.get(idx) for idx in output]
    words = [word for word in words if word not in (',', '.', '<end>')]
    sentence = " ".join(words)
    
    return sentence

def get_prediction(data_loader, df):
    print("="*20)
    orig_image, image, cat_onehot, seq_onehot, caption= next(iter(data_loader))
    plt.imshow(orig_image.squeeze())
    plt.title(caption)
    plt.show()
    file_name = df[df["description"] == caption[0]]["file_name"].values[0]
    image = image.to(device)
    #display(test_df[(test_df["file_name"]==file_name)])
    for sentence_id in range(6):
        seq_onehot = torch.Tensor(data_loader.dataset.onehot_seq_encoder.transform(np.array([sentence_id], 
                                                                            dtype=object).reshape(-1, 1)).toarray()).long()
        seq_onehot.to(device)
        print(sentence_id)
        onehot = torch.cat((cat_onehot.view(1,-1), 
                            seq_onehot.view(1,-1)), 1).type('torch.FloatTensor').view(1,-1).to(device)
        
        features = encoder(image)
        output, atten_weights = decoder.greedy_search(features, onehot)    
        sentence = clean_sentence(output,data_loader)
        print(df[(df["file_name"]==file_name) & (df["sentence_index"]==sentence_id)]["description"].values)
        print(sentence)
        #print(sentence.replace("<unk>", ""))


In [ ]:

# %%
for i in range(1):
    get_prediction(test_data_loader, test_df)